In [630]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy import*
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

In [631]:
def PsFromTsd(T_sd):
    #T_sd basert i s, d=desired. Gir konfigurasjonen vi vil ha end-effector framen, B, i.
    #B, og derav og M, er lik som i DH
    #s er plassert nederst på roboten med positiv z oppover, altså ikke som i DH. Bør kanskje endres til å være lik DH 
    #T_sb = np.array([[0,0,-1,0],
    #                [0,1,0,0],
    #                 [1,0,0,0],
    #                 [0,0,0,1]])
    #T_sb = T_sd@T_db
    P_d = np.array([0,0,80,1])
    P_s = T_sd@P_d

    return P_s

def exp3(omega, theta):
    omega = skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def skew(v):
    return Matrix([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
                    
def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3,:3] = exp3(twist[:3], theta)
    T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
              (theta-sin(theta)) * omega * omega) * v
    return T


def rotX(alfa_im1):
    Rx = sp.eye(4)
    Rx[1,1] =    sp.cos(alfa_im1)
    Rx[1,2] =   -sp.sin(alfa_im1)
    Rx[2,1] =    sp.sin(alfa_im1)
    Rx[2,2] =    sp.cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = sp.eye(4)
    Rz[0,0] =    sp.cos(alfa_im1)
    Rz[0,1] =   -sp.sin(alfa_im1)
    Rz[1,0] =    sp.sin(alfa_im1)
    Rz[1,1] =    sp.cos(alfa_im1)
    return Rz

def rotY(alfa_im1):
    Ry = sp.eye(4)
    Ry[0,0] =    sp.cos(alfa_im1)
    Ry[0,2] =   sp.sin(alfa_im1)
    Ry[2,0] =    -sp.sin(alfa_im1)
    Ry[2,2] =    sp.cos(alfa_im1)
    return Ry

def eulerZYZ(R):
    thetas = []
    thetas[0] = atan2(-round(R[2,1],6), round(R[2,0],6)+0.000001)
    thetas[1] = atan2(round(sqrt(R[2,0]**2 + R[2,1]**2),6), round(R[2,2],6)+0.000001)
    thetas[2] = atan2(round(R[1,2],6), round(R[0,2],6)+0.000001)

    


In [681]:
S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,25])
S4 = np.array([-1,0,0,0,-890,0])
S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
print(Slist)

M = np.array([[0,0,-1,525],
              [0,1,0,0],
              [1,0,0,890],
              [0,0,0,1]])

thetas = [0,0,0,0,0,0]

[[   0    0    0   -1    0   -1]
 [   0    1    1    0    1    0]
 [  -1    0    0    0    0    0]
 [   0 -400 -855    0 -890    0]
 [   0    0    0 -890    0 -890]
 [   0   25   25    0  445    0]]


In [703]:

#OPPGAVE 3.2

#Her endres T_sd for å teste andre gyldige konfigurasjoner:
#T_sd = M
thetas_gen = np.array([0,0,0,1,np.pi/2-0.01,np.pi/2-0.1])
T_sd = mr.FKinSpace(M,Slist,thetas_gen)
print("T_sd\n", T_sd)

P_s = PsFromTsd(T_sd)
print("Ps", P_s)

#theta1
print("atan2 arg:",P_s[1], P_s[0])
thetas[0] = -atan2(P_s[1],P_s[0]) #minus foran fordi vinkelen er definert andre vei
print(N(thetas[0]), thetas_gen[0])

#theta2 and theta3
a = np.sqrt(420**2+35**2)
c = 455
P_merket = np.array([P_s[0],P_s[1],P_s[2]-400])
b = np.sqrt((np.sqrt(P_merket[0]**2+P_merket[1]**2)-25)**2 + P_merket[2]**2)
print("P_merket:", P_merket)

psi = np.arccos(420/a) #Se notatbok
phi = atan2(P_merket[2], sqrt(P_merket[0]**2 + P_merket[1]**2)-25)

alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
beta = np.arccos((a**2+c**2-b**2)/(2*a*c))
print("beta:", beta)
print("psi:", psi)
print("phi:", N(phi))
print("alpha", alpha)

thetas[1] = np.pi/2 - (alpha + phi)
print("that2", thetas[1])
thetas[2] = np.pi/2 - (beta-psi)

#theta4,5,6
#R_d = T_sd[:3][:3]
#Vi har ZYZ euler angles i stedet for ZYX som gitt i forelesning

T1 = exp6(S1, -thetas[0])
T2 = exp6(S2, -thetas[1])
T3 = exp6(S3, -thetas[2])


R = T3@T2@T1@T_sd@np.linalg.inv(M) #R er definert i space frame? 
#R = N(rotZ(2)@rotY(2)@rotZ(0))

#print(N(R))

#thetas[3] = atan2(round(R[1,0],6), round(R[0,0],6)+0.000001)
thetas[3] = atan2(round(R[1,2],6), round(R[0,2],6)+0.000000001)


#thetas[4] = atan2(round(-R[2,0],6), round(sqrt(R[0,0]**2 + R[1,0]**2),6))
thetas[4] = atan2(round(sqrt(R[2,0]**2 + R[2,1]**2),6), round(R[2,2],6)+0.000000001)


#thetas[5] = atan2(round(R[2,1],6), round(R[2,2],6)+0.000001)
thetas[5] = np.pi + atan2(-R[2,1], R[2,0]) #+np.pi funker av en eller annen grunn

print("R[2,1], R[2,0]",R[2,1],R[2,0])


for i in thetas:
    print(N(i))

T_sd
 [[  0.0998  -0.995   -0.01   445.8   ]
 [  0.5384   0.0456   0.8414 -67.3143]
 [ -0.8367  -0.0894   0.5403 846.778 ]
 [  0.       0.       0.       1.    ]]
Ps [445.  -0. 890.   1.]
atan2 arg: -1.9895196601282805e-13 444.99999999999994
4.47083069691748e-16 0.0
P_merket: [445.  -0. 490.]
beta: 1.6539375586833374
psi: 0.08314123188844062
phi: 0.862170054667226
alpha 0.7086262721276707
that2 -4.44089209850063e-16
R[2,1], R[2,0] -0.0893828642709648 -0.540275290977972
4.47083069691748e-16
-4.44089209850063e-16
-2.22044604925031e-16
1.38747660549498
2.56207678538317
6.11923088438148


In [756]:
#testing av analytisk løsning:
thetas_num = np.zeros(6)
thetas_deg = np.zeros(6)
for i in range(0,6):
    thetas_num[i] = N(thetas[i])
    thetas_deg[i] = np.rad2deg(thetas_num[i])

print(thetas_deg, np.rad2deg(thetas_gen))
print(thetas_num, thetas_gen)

T_thetas = mr.FKinSpace(M,Slist,thetas_num)
P_reached = PsFromTsd(T_thetas)
print(P_reached, P_s)
np.set_printoptions(precision=4)

fk_test = exp6(S4,thetas_num[3])@exp6(S5,thetas_num[4])@exp6(S6,thetas_num[5])@M #Roterer M om S gitt i space frame
R_test = rotZ(thetas_num[3])@rotY(thetas_num[4])@rotZ(thetas_num[5]) #Denne bør bli samme som R i cellen over. Det blir den, men POE med samme vinklene gir ikke riktig rotasjon. 

#R, R_test
T_sd, T_thetas, R_test, fk_test, R


[  0.      -0.      -0.      79.4966 146.7962 350.6061] [ 0.      0.      0.     57.2958 89.427  84.2704]
[ 0.     -0.     -0.      1.3875  2.5621  6.1192] [0.     0.     0.     1.     1.5608 1.4708]
[445.   0. 890.   1.] [445.  -0. 890.   1.]


(array([[  0.0998,  -0.995 ,  -0.01  , 445.8   ],
        [  0.5384,   0.0456,   0.8414, -67.3143],
        [ -0.8367,  -0.0894,   0.5403, 846.778 ],
        [  0.    ,   0.    ,   0.    ,   1.    ]]),
 array([[  0.5403,   0.0894,   0.8367, 378.0618],
        [ -0.8414,   0.0456,   0.5384, -43.0754],
        [  0.01  ,  -0.995 ,   0.0998, 882.0138],
        [  0.    ,   0.    ,   0.    ,   1.    ]]),
 Matrix([
 [0.0100004073161593,   -0.99495445154866, 0.0998280080789753, 0],
 [-0.841429026849547,  0.0455669676456226,  0.538442981414567, 0],
 [-0.540275100877817, -0.0893828328208938, -0.836727867689595, 0],
 [                 0,                   0,                  0, 1]]),
 Matrix([
 [ 0.540275100877817, 0.0893828328208938,  0.836727867689595,  378.061770584832],
 [-0.841429026849547, 0.0455669676456226,  0.538442981414567, -43.0754385131652],
 [0.0100004073161593,  -0.99495445154866, 0.0998280080789753,  882.013759353682],
 [                 0,                  0,                  0

In [762]:
sy = np.array([0,1,0,0,0,0])
sz = np.array([0,0,1,0,0,0])
T1 = exp6(S1, thetas[0])
T2 = exp6(S2, thetas[1])
T3 = exp6(S3, thetas[2])

R_test2 = T1@T2@T3@M@R_test




fk_test = exp6(S4,np.pi/2)@M
R_test = M@rotZ(np.pi/2)
print(M)
print(T_sd)

N(R_test2), N(fk_test)

[[  0   0  -1 525]
 [  0   1   0   0]
 [  1   0   0 890]
 [  0   0   0   1]]
[[  0.0998  -0.995   -0.01   445.8   ]
 [  0.5384   0.0456   0.8414 -67.3143]
 [ -0.8367  -0.0894   0.5403 846.778 ]
 [  0.       0.       0.       1.    ]]


⎛⎡3.85850729734381e-16            1.0           6.66133814775094e-16    -365.0
⎜⎢                                                                            
⎜⎢         1.0           -3.85850729734381e-16  -2.97817150735124e-31  1.63185
⎜⎢                                                                            
⎜⎢-4.07889322034067e-32  6.66133814775094e-16           -1.0                  
⎜⎢                                                                            
⎝⎣          0                      0                      0                   

00000000001  ⎤  ⎡         0                     0            -1.0  525.0⎤⎞
             ⎥  ⎢                                                       ⎥⎟
320437488e-13⎥  ⎢        1.0           1.11022302462516e-16   0      0  ⎥⎟
             ⎥, ⎢                                                       ⎥⎟
1415.0       ⎥  ⎢1.11022302462516e-16          -1.0           0    890.0⎥⎟
             ⎥  ⎢                                                      